In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [2]:
def request_backoff(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    }
    
    time.sleep(random.uniform(3,4))
    max_retries = 6
    delay_base = 3 #initial delay
    
    for retry_count in range (max_retries):
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            return response
        elif response.status_code == 429: #rate limited
            retry_delay = delay_base * (2 ** retry_count) + random.uniform(0, 1)  # Exponential backoff with random factor
            print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            
        elif response.status_code == 400: #no more jobs
            break
            
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            
    print("Max retries reached. Aborting.")
    return None
        
            
            
    

In [3]:
title = "Python Developer"
location = "Toronto"

In [4]:
start = 0
id_list = []
success = True

while success:
    
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&position=1&pageNum=0&start={start}"
    
    try:
        response = request_backoff(list_url)
        list_data = response.text
        list_soup = BeautifulSoup(list_data, "html.parser")
        page_jobs = list_soup.find_all("li")

    except Exception as e:
        page_jobs = None
        
    if not page_jobs:
        
        print("No more jobs")        
        break #exit the loop if no jobs are found
    
    for job in page_jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        if base_card_div and 'data-entity-urn' in base_card_div.attrs:
            job_id = base_card_div.get('data-entity-urn').split(":")[3]
            print(job_id)
            id_list.append(job_id)
            start += 1
            
    print(f"new start: {start}")
        
        
    
    

Rate limit exceeded. Retrying in 3.1895391895104224 seconds...
Rate limit exceeded. Retrying in 6.4558754239743985 seconds...
Rate limit exceeded. Retrying in 12.134214444128261 seconds...
Rate limit exceeded. Retrying in 24.66886602101131 seconds...
Rate limit exceeded. Retrying in 48.34531871966906 seconds...
3844988569
3844984812
3761549941
3847723088
3764242184
3844986750
3844984527
3789900117
3844986412
3784890058
3824331371
3848984345
3845807524
3818361840
3818730423
3818361829
3793506685
3835928708
3828063931
3850374196
3842087330
3818813473
3783999274
3847902923
3839588912
new start: 25
Rate limit exceeded. Retrying in 3.525905164555298 seconds...
Rate limit exceeded. Retrying in 6.334276171408364 seconds...
3851023196
3746835882
3840172578
3852158395
3782003334
3835566820
3847632005
3775483217
3848954711
3590300623
3777162588
3818807800
3844958156
3818809894
3853127206
3848961355
3782104508
3848715241
3843336654
3817447419
3757481767
3849890208
3745346455
3784867928
3818819127

In [5]:
#remove duplicates
id_list = list(set(id_list))

In [6]:
len(id_list)

117

In [7]:
job_list = []

for job_id in id_list:

    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    
    try:

        response = request_backoff(job_url)

        if response.status_code == 200:

            job_soup = BeautifulSoup(response.text, 'html.parser')

            job_post = {}  # Initialize 'job_post' as an empty dictionary before using it

            # Attempt to fetch company name
            try:
                job_post["company"] = job_soup.find("div", {"class": "top-card-layout__card"}).find("a").find("img").get('alt')
            except:
                job_post["company"] = None

            # Attempt to fetch job title
            try:
                job_post["job-title"] = job_soup.find("div", {"class": "top-card-layout__entity-info"}).find("a").text.strip()  # Corrected to use job_soup
            except:
                job_post["job-title"] = None

            # Attempt to fetch level
            try:
                job_post["level"] = job_soup.find("ul", {"class": "description__job-criteria-list"}).find("li").text.replace("Seniority level", "").strip()  # Corrected to use job_soup
            except:
                job_post["level"] = None

            try:
                job_post["description"] = job_soup.find("div", {"class": "decorated-job-posting__details"}).text.strip()
            except:
                job_post["description"] = None

            try:
                job_post["url"] = job_url
            except:
                job_post["url"] = None

            job_list.append(job_post)
            print(job_post)
            print(f"Successfully fetched data for job ID {job_id}")

        else:
            print(f"Failed to fetch data for job ID {job_id}. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred while fetching data for job ID {job_id}: {e}")

{'company': 'Epsilon Solutions Ltd.', 'job-title': 'Python Developer', 'level': 'Entry level', 'description': 'Role: Python DeveloperJob Description" 5+ years of experience of extensive hands-on development, ideally in Saas companies. Expertise in Python programing language excellent coding skills (should be able to solve coding problems during interview efficiently) excellent communication skills (good CV, be able to clearly explain experience in detail) Experience with security, Identity management, Authorization is a plus. Experience working in Agile teams with CI/CD pipelines and automated testing" Python developer, Experience with security, Identity management, Authorization.\n\n\n\n        \n            Show more\n          \n\n          \n\n\n\n        \n            Show less\n          \n\n          \n\n \n\n\n\n\n            Seniority level\n          \n\n            Entry level\n          \n\n\n\n            Employment type\n          \n\n            Contract\n          \n\n\

{'company': 'Scotiabank', 'job-title': 'Senior Software Engineer | Python Developer', 'level': 'Not Applicable', 'description': 'Requisition ID: 189673Join a purpose driven winning team, committed to results, in an inclusive and high-performing culture.The TeamGlobal Banking and Markets Engineering (GBME) is the fast-moving, award-winning technology engine that powers Scotiabank’s Corporate, Investment Banking and Capital Markets businesses.GBME is searching for Software Engineers who are continuous learners and are eager to boost capabilities of capital markets products and analytics platforms.The successful candidate will have a strong technical and engineering mindset. You will work closely with business teams and cross functional engineering leaders to develop and implement market leading solutions for the various projects within GBME.Is this role right for you? In this role you will:Work closely with end users, and business teams to understand business needs, conduct gap analysis 

Rate limit exceeded. Retrying in 3.1831354846403643 seconds...
{'company': 'Tundra Technical Solutions', 'job-title': 'Python Developer (Lead) (contract)', 'level': 'Mid-Senior level', 'description': 'Looking for 4 engineers to complete a pod of 6 engineers to build the pipeline b/w sub-ledgers and data fabric. Should be built in AWS, able to send request to API, gather the info, and publish the info to data fabric. Resources with Python exp with pandas, Python with requests, AWS Airflow Dag, and SQL will be required. 7 resources will be dispersed over multiple sib-ledgers.Our client is home to innovative thinkers who are vital to our success, and we are proud of the diverse and vibrant teams we have established worldwide. We are committed to hiring ambitious, driven engineers and creating an environment where our employees can thrive and grow through recognition and reward of their dedication as they progress within the company.The OpportunityWe are looking for Mid-Senior Software Eng

{'company': 'Zortech Solutions', 'job-title': 'Python Engineering-Junio Level', 'level': 'Entry level', 'description': 'Role: Python Engineering-Junio LevelLocation: Remote/CanadaDuration: 6+ MonthsRequiredJob Description:1-2 years of experience as a software developer.1 year of experience with Python.Experience with a cloud platform such as AWS, Google Cloud or Azure.Nice To HaveExperience with AWS services such as ECS and Lambda.Knowledge of development principles such as OOP.Experience with relational or non-relational databases such as MySQL, Postgres, Mongo and Dynamo.Experience with a CI / CD platform such as CircleCI, Travis, GitHub Actions or ADO.\n\n\n\n        \n            Show more\n          \n\n          \n\n\n\n        \n            Show less\n          \n\n          \n\n \n\n\n\n\n            Seniority level\n          \n\n            Entry level\n          \n\n\n\n          Employment type\n        \n\n          Contract\n        \n\n\n\n            Job function\n     

{'company': 'Tundra Technical Solutions', 'job-title': 'Software Developer (Python) - 3 (contract)', 'level': 'Mid-Senior level', 'description': "Story Behind The NeedBusiness Group: Enterprise Stress TestingProject: Scotia has launched a platform modernization project. This role will assist with implementing models, updating analytics workflows to use new models, and migrating models and analytics processes to new infrastructure. Possibility of working on other project as they come up.Candidate Value Proposition: The candidate will have the opportunity to work with one of Canada’s top 5 banks along side a dynamic team.Typical Day In Role Analyze highly complex business requirements and generate technical solutions Leverage industry best practices to design or redesign complex software components and applications Implement production-ready code and assist with integration with larger IT workflows Assure quality security and compliance requirements are met for supported area Be flexible

...

Rate limit exceeded. Retrying in 3.5798986388224323 seconds...
Rate limit exceeded. Retrying in 6.65853179865763 seconds...
Rate limit exceeded. Retrying in 12.272626143703654 seconds...
Rate limit exceeded. Retrying in 24.64343337888758 seconds...
Rate limit exceeded. Retrying in 48.367054592519416 seconds...
{'company': 'CI Financial', 'job-title': 'Python Developer', 'level': 'Mid-Senior level', 'description': 'Direct message the job poster from CI Financial\n\n\n\n\n\n            \n        Alexandra Colangelo\n\n\n\n\n              \n        Full Lifecycle Technical Recruiting | Diversity Focused | Talent Management\n      \n            \n\n \n\n\n\n\n\n\n\n\n\n\nSoftware Developer At CI, we see a great place to work as one that is a safe place for everyone to have a voice, where people are empowered to take ownership over meaningful work, where there is an opportunity to grow through stretching themselves, where they can work on innovative products and projects, and where employee

Rate limit exceeded. Retrying in 3.803054684534514 seconds...
Rate limit exceeded. Retrying in 6.631461070864402 seconds...
Rate limit exceeded. Retrying in 12.170038781415705 seconds...
Rate limit exceeded. Retrying in 24.65512621037439 seconds...
Rate limit exceeded. Retrying in 48.256946418161995 seconds...
Rate limit exceeded. Retrying in 96.97351621014563 seconds...
Max retries reached. Aborting.
An error occurred while fetching data for job ID 3764242184: 'NoneType' object has no attribute 'status_code'
{'company': 'Cockroach Labs', 'job-title': 'Software Engineer, Backend (Cloud Platforms) - Toronto', 'level': 'Entry level', 'description': "Databases are the beating heart of every business in the world.Cockroach Labs is the team behind CockroachDB, the most highly evolved cloud-native, distributed SQL database on the planet. We created CockroachDB and our self-service, fully managed cloud offerings of CockroachDB (Dedicated and Serverless) to deliver the ability to build and sca

{'company': 'Robinhood', 'job-title': 'Web Developer, Crypto', 'level': 'Entry level', 'description': "Join a leading fintech company that’s democratizing finance for all.Robinhood was founded on a simple idea: that our financial markets should be accessible to all. With customers at the heart of our decisions, Robinhood is lowering barriers and providing greater access to financial information. Together, we are building products and services that help create a financial system everyone can participate in.As we continue to build...We’re seeking curious, growth minded thinkers to help shape our vision, structures and systems; playing a key-role as we launch into our ambitious future. If you’re invigorated by our mission, values, and drive to change the world — we’d love to have you apply.About the team + role Robinhood crypto recently launched support for trading in Europe, and the team which built this is expanding and hiring a Web engineer. You’ll get to work on the core product featu

{'company': 'Diverse Lynx', 'job-title': 'QA Engineer with(Python + Robot Framework)', 'level': 'Entry level', 'description': 'Job title: QA (Python + Robot Framework)Job location: Toronto, ON,(Hybrid)Job type: Contract6+ years of experience as Quality AnalystHands On candidate in SQLUnixETLPythonRobot (Optional) andGood QE process knowledge and communication skills.Diverse Lynx LLC is an Equal Employment Opportunity employer. All qualified applicants will receive due consideration for employment without any discrimination. All applicants will be evaluated solely on the basis of their ability, competence and their proven capability to perform the functions outlined in the corresponding role. We promote and support a diverse workforce across all levels in the company.\n\n\n\n        \n            Show more\n          \n\n          \n\n\n\n        \n            Show less\n          \n\n          \n\n \n\n\n\n\n            Seniority level\n          \n\n            Entry level\n          

{'company': 'Della Connect', 'job-title': 'Backend Developer', 'level': 'Entry level', 'description': 'Our company is a rapidly growing technology firm that specializes in developing innovative software solutions for clients across various industries. We take pride in building software that is efficient, scalable, secure, and user-friendly. Our goal is to make a significant impact in the market through cutting-edge technology.We are looking for a Backend Developer to join our team. As a Backend Developer, you will be responsible for designing, developing, and maintaining applications using Nods.js and Google Cloud. The ideal candidate will have a strong background in software development, a solid understanding of web development principles, and a proven track record of working with Node.js / Typescript.Note: Only candidates located within a 3-hour time zone from Toronto, Canada will be considered for this role.What You’ll OwnWriting clean, maintainable, and well-documented code using N

In [8]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,company,job-title,level,description,url
0,Epsilon Solutions Ltd.,Python Developer,Entry level,"Role: Python DeveloperJob Description"" 5+ year...",https://www.linkedin.com/jobs-guest/jobs/api/j...
1,Zortech Solutions,"Data Engineer-SQL, Python, ETL- Canada",Entry level,"Role: Data Engineer-SQL, Python, ETLLocation: ...",https://www.linkedin.com/jobs-guest/jobs/api/j...
2,Boomerang,Software Engineer (Full-Stack),Employment type\n \n\n Full-time,Direct message the job poster from Boomerang\n...,https://www.linkedin.com/jobs-guest/jobs/api/j...
3,Phoenix (YC W21),Full-Stack Developer,Entry level,What we're looking forWe are looking for a ful...,https://www.linkedin.com/jobs-guest/jobs/api/j...
4,Scotiabank,Senior Software Engineer | Python Developer,Not Applicable,Requisition ID: 189673Join a purpose driven wi...,https://www.linkedin.com/jobs-guest/jobs/api/j...
...,...,...,...,...,...
108,Zortech Solutions,Python Developer,Entry level,Role: Python DeveloperLocation: Toronto/Remote...,https://www.linkedin.com/jobs-guest/jobs/api/j...
109,Seven Hills Group Technologies Inc.,Python Developer with AWS.,Entry level,"Hi,I am Santhi swaroop, With Seven Hills Group...",https://www.linkedin.com/jobs-guest/jobs/api/j...
110,ClifyX,Full Stack Developer,Entry level,Reg ID : 9253432 Role/Skill : Full Stack Devel...,https://www.linkedin.com/jobs-guest/jobs/api/j...
111,Della Connect,Backend Developer,Entry level,Our company is a rapidly growing technology fi...,https://www.linkedin.com/jobs-guest/jobs/api/j...


In [9]:
jobs_df.to_csv('Toronto_Python_Developer.csv', index=False)